<a href="https://colab.research.google.com/github/shumph10/Housing_Assistance_Analysis/blob/sarah_preprocessing/Sup_ML_Modelipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.2 MB/s 


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
import keras_tuner
from tensorflow import keras
import kerastuner as kt
import os
from tensorflow.keras.callbacks import ModelCheckpoint


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if __name__ == '__main__':


In [2]:
#  Import and read the charity_data.csv.
rent_1722_df = pd.read_csv("sample_data/Sec8_FMR_17&22.csv")
rent_1722_df.head()

,County_Name,ELI_2_17,ELI_2_22,Metro_Area_Name,State_Alpha,_id,county,county_town_name,fmr0_17,fmr1_17,...,l50_2_17,l50_2_22,l80_2_17,l80_2_22,median2017,median2022,metro,metro_code,state,state_name
0,Autauga County,16240,18310,"Montgomery, AL MSA",AL,100199999,1,Autauga County,587,682,...,23900,30200,38200,48350,59700,75500,1,METRO33860M33860,1,Alabama
1,Baldwin County,16240,20550,"Daphne-Fairhope-Foley, AL MSA",AL,100399999,3,Baldwin County,807,847,...,24600,34200,39400,54750,61500,85500,1,METRO19300M19300,1,Alabama
2,Barbour County,16240,18310,"Barbour County, AL",AL,100599999,5,Barbour County,501,505,...,18450,24650,29550,39450,46100,46400,0,NCNTY01005N01005,1,Alabama
3,Bibb County,16240,20400,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,100799999,7,Bibb County,665,751,...,25250,33950,40400,54300,63100,84800,1,METRO13820M13820,1,Alabama
4,Blount County,16240,20400,"Birmingham-Hoover, AL HUD Metro FMR Area",AL,100999999,9,Blount County,665,751,...,25250,33950,40400,54300,63100,84800,1,METRO13820M13820,1,Alabama


In [3]:
rent_1722_df.columns

Index(['County_Name', 'ELI_2_17', 'ELI_2_22', 'Metro_Area_Name', 'State_Alpha',
       '_id', 'county', 'county_town_name', 'fmr0_17', 'fmr1_17', 'fmr2_17',
       'fmr3_17', 'fmr4_17', 'fmr_0_22', 'fmr_1_22', 'fmr_2_22', 'fmr_3_22',
       'fmr_4_22', 'l50_2_17', 'l50_2_22', 'l80_2_17', 'l80_2_22',
       'median2017', 'median2022', 'metro', 'metro_code', 'state',
       'state_name'],
      dtype='object')

In [5]:
#reorder columns
rent_1722_df = rent_1722_df[['County_Name', 'county', 'county_town_name', 'state',
       'state_name', 'State_Alpha', 'Metro_Area_Name', 'ELI_2_17', 'ELI_2_22', 'median2017', 'median2022',
       '_id', 'fmr0_17', 'fmr1_17', 'fmr2_17',
       'fmr3_17', 'fmr4_17', 'fmr_0_22', 'fmr_1_22', 'fmr_2_22', 'fmr_3_22',
       'fmr_4_22', 'l50_2_17', 'l50_2_22', 'l80_2_17', 'l80_2_22',
       'metro', 'metro_code']]
rent_1722_df.head()

,County_Name,county,county_town_name,state,state_name,State_Alpha,Metro_Area_Name,ELI_2_17,ELI_2_22,median2017,...,fmr_1_22,fmr_2_22,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,metro,metro_code
0,Autauga County,1,Autauga County,1,Alabama,AL,"Montgomery, AL MSA",16240,18310,59700,...,764,914,1156,1494,23900,30200,38200,48350,1,METRO33860M33860
1,Baldwin County,3,Baldwin County,1,Alabama,AL,"Daphne-Fairhope-Foley, AL MSA",16240,20550,61500,...,777,1017,1348,1715,24600,34200,39400,54750,1,METRO19300M19300
2,Barbour County,5,Barbour County,1,Alabama,AL,"Barbour County, AL",16240,18310,46100,...,536,705,871,980,18450,24650,29550,39450,0,NCNTY01005N01005
3,Bibb County,7,Bibb County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,820,943,1220,1316,25250,33950,40400,54300,1,METRO13820M13820
4,Blount County,9,Blount County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,820,943,1220,1316,25250,33950,40400,54300,1,METRO13820M13820


Adding 0's before state and county codes in order to combine them

In [16]:
#adding 0's in from of state and county id's
rent_df = rent_1722_df.copy()
rent_df['county'] = rent_df['county'].map(lambda x: f'{x:0>3}')
rent_df.head()

,County_Name,county,county_town_name,state,state_name,State_Alpha,Metro_Area_Name,ELI_2_17,ELI_2_22,median2017,...,fmr_1_22,fmr_2_22,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,metro,metro_code
0,Autauga County,001,Autauga County,1,Alabama,AL,"Montgomery, AL MSA",16240,18310,59700,...,764,914,1156,1494,23900,30200,38200,48350,1,METRO33860M33860
1,Baldwin County,003,Baldwin County,1,Alabama,AL,"Daphne-Fairhope-Foley, AL MSA",16240,20550,61500,...,777,1017,1348,1715,24600,34200,39400,54750,1,METRO19300M19300
2,Barbour County,005,Barbour County,1,Alabama,AL,"Barbour County, AL",16240,18310,46100,...,536,705,871,980,18450,24650,29550,39450,0,NCNTY01005N01005
3,Bibb County,007,Bibb County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,820,943,1220,1316,25250,33950,40400,54300,1,METRO13820M13820
4,Blount County,009,Blount County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,820,943,1220,1316,25250,33950,40400,54300,1,METRO13820M13820


In [17]:
#check end of df
rent_df.tail()

,County_Name,county,county_town_name,state,state_name,State_Alpha,Metro_Area_Name,ELI_2_17,ELI_2_22,median2017,...,fmr_1_22,fmr_2_22,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,metro,metro_code
4751,Yabucoa Municipio,151,Yabucoa Municipio,72,Puerto Rico,PR,"San Juan-Guaynabo, PR HUD Metro FMR Area",7750,7650,26600,...,466,538,719,868,12900,12750,20600,20400,1,METRO41980MM7440
4752,Yauco Municipio,153,Yauco Municipio,72,Puerto Rico,PR,"Yauco, PR MSA",6050,5950,17000,...,370,422,572,574,10100,9850,16150,15800,1,METRO49500M49500
4753,St. Croix,010,St. Croix,78,Virgin Islands,VI,"St. Croix Island, VI",11950,15950,46700,...,886,1082,1338,1467,19900,26600,31800,42500,0,NCNTY78010N78010
4754,St. John,020,St. John,78,Virgin Islands,VI,"St. John Island, VI",16550,23800,56300,...,1368,1700,2101,2305,27550,39650,44050,63450,0,NCNTY78020N78020
4755,St. Thomas,030,St. Thomas,78,Virgin Islands,VI,"St. Thomas Island, VI",14750,17550,52400,...,1001,1253,1549,1699,24600,29200,39300,46750,0,NCNTY78030N78030


In [18]:
rent_df.dtypes

County_Name         object
county              object
county_town_name    object
state                int64
state_name          object
State_Alpha         object
Metro_Area_Name     object
ELI_2_17             int64
ELI_2_22             int64
median2017           int64
median2022           int64
_id                  int64
fmr0_17              int64
fmr1_17              int64
fmr2_17              int64
fmr3_17              int64
fmr4_17              int64
fmr_0_22             int64
fmr_1_22             int64
fmr_2_22             int64
fmr_3_22             int64
fmr_4_22             int64
l50_2_17             int64
l50_2_22             int64
l80_2_17             int64
l80_2_22             int64
metro                int64
metro_code          object
dtype: object

In [19]:
rent_df['state_county_code'] = rent_df['state'].map(str) + rent_df['county'].map(str)
rent_df.head()

,County_Name,county,county_town_name,state,state_name,State_Alpha,Metro_Area_Name,ELI_2_17,ELI_2_22,median2017,...,fmr_2_22,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,metro,metro_code,state_county_code
0,Autauga County,001,Autauga County,1,Alabama,AL,"Montgomery, AL MSA",16240,18310,59700,...,914,1156,1494,23900,30200,38200,48350,1,METRO33860M33860,1001
1,Baldwin County,003,Baldwin County,1,Alabama,AL,"Daphne-Fairhope-Foley, AL MSA",16240,20550,61500,...,1017,1348,1715,24600,34200,39400,54750,1,METRO19300M19300,1003
2,Barbour County,005,Barbour County,1,Alabama,AL,"Barbour County, AL",16240,18310,46100,...,705,871,980,18450,24650,29550,39450,0,NCNTY01005N01005,1005
3,Bibb County,007,Bibb County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,943,1220,1316,25250,33950,40400,54300,1,METRO13820M13820,1007
4,Blount County,009,Blount County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,943,1220,1316,25250,33950,40400,54300,1,METRO13820M13820,1009


In [20]:
rent_df.dtypes

County_Name          object
county               object
county_town_name     object
state                 int64
state_name           object
State_Alpha          object
Metro_Area_Name      object
ELI_2_17              int64
ELI_2_22              int64
median2017            int64
median2022            int64
_id                   int64
fmr0_17               int64
fmr1_17               int64
fmr2_17               int64
fmr3_17               int64
fmr4_17               int64
fmr_0_22              int64
fmr_1_22              int64
fmr_2_22              int64
fmr_3_22              int64
fmr_4_22              int64
l50_2_17              int64
l50_2_22              int64
l80_2_17              int64
l80_2_22              int64
metro                 int64
metro_code           object
state_county_code    object
dtype: object

In [21]:
rent_df['state_county_code'] = pd.to_numeric(rent_df['state_county_code'], errors='coerce')
rent_df['county'] = pd.to_numeric(rent_df['county'], errors='coerce')
rent_df.dtypes

County_Name          object
county                int64
county_town_name     object
state                 int64
state_name           object
State_Alpha          object
Metro_Area_Name      object
ELI_2_17              int64
ELI_2_22              int64
median2017            int64
median2022            int64
_id                   int64
fmr0_17               int64
fmr1_17               int64
fmr2_17               int64
fmr3_17               int64
fmr4_17               int64
fmr_0_22              int64
fmr_1_22              int64
fmr_2_22              int64
fmr_3_22              int64
fmr_4_22              int64
l50_2_17              int64
l50_2_22              int64
l80_2_17              int64
l80_2_22              int64
metro                 int64
metro_code           object
state_county_code     int64
dtype: object

In [22]:
rent_df.tail()

,County_Name,county,county_town_name,state,state_name,State_Alpha,Metro_Area_Name,ELI_2_17,ELI_2_22,median2017,...,fmr_2_22,fmr_3_22,fmr_4_22,l50_2_17,l50_2_22,l80_2_17,l80_2_22,metro,metro_code,state_county_code
4751,Yabucoa Municipio,151,Yabucoa Municipio,72,Puerto Rico,PR,"San Juan-Guaynabo, PR HUD Metro FMR Area",7750,7650,26600,...,538,719,868,12900,12750,20600,20400,1,METRO41980MM7440,72151
4752,Yauco Municipio,153,Yauco Municipio,72,Puerto Rico,PR,"Yauco, PR MSA",6050,5950,17000,...,422,572,574,10100,9850,16150,15800,1,METRO49500M49500,72153
4753,St. Croix,10,St. Croix,78,Virgin Islands,VI,"St. Croix Island, VI",11950,15950,46700,...,1082,1338,1467,19900,26600,31800,42500,0,NCNTY78010N78010,78010
4754,St. John,20,St. John,78,Virgin Islands,VI,"St. John Island, VI",16550,23800,56300,...,1700,2101,2305,27550,39650,44050,63450,0,NCNTY78020N78020,78020
4755,St. Thomas,30,St. Thomas,78,Virgin Islands,VI,"St. Thomas Island, VI",14750,17550,52400,...,1253,1549,1699,24600,29200,39300,46750,0,NCNTY78030N78030,78030


In [23]:
waits_util_df = pd.read_csv("sample_data/county_data_merged.csv")
waits_util_df.head()

,_id,ave_util_allow_17,ave_util_allow_21,code_17,code_21,entities,hh_income_17,hh_income_21,months_from_movein_17,months_from_movein_21,months_waiting_17,months_waiting_21,name_17,name_21,pct_utility_allow_17,pct_utility_allow_21,program_label_17,program_label_21
0,62efb19aaf83d320cc1839f2,162,189,1001,1001,AL Autauga County 01001,13903,14185,87,107,7,-1,Autauga County,Autauga County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
1,62efb19aaf83d320cc1839f3,195,211,1003,1003,AL Baldwin County 01003,13529,14275,102,116,52,9,Baldwin County,Baldwin County,96,98,Housing Choice Vouchers,Housing Choice Vouchers
2,62efb19aaf83d320cc1839f4,178,192,1005,1005,AL Barbour County 01005,12884,13396,72,89,6,6,Barbour County,Barbour County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
3,62efb19aaf83d320cc1839f5,120,158,1009,1009,AL Blount County 01009,12686,14428,82,92,6,9,Blount County,Blount County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
4,62efb19aaf83d320cc1839f6,151,195,1013,1013,AL Butler County 01013,11408,12531,53,65,5,7,Butler County,Butler County,100,100,Housing Choice Vouchers,Housing Choice Vouchers


In [28]:
waits_util_df.rename(columns={"code_17": "state_county_code"}, inplace=True)
waits_util_df.head()

,_id,ave_util_allow_17,ave_util_allow_21,state_county_code,code_21,entities,hh_income_17,hh_income_21,months_from_movein_17,months_from_movein_21,months_waiting_17,months_waiting_21,name_17,name_21,pct_utility_allow_17,pct_utility_allow_21,program_label_17,program_label_21
0,62efb19aaf83d320cc1839f2,162,189,1001,1001,AL Autauga County 01001,13903,14185,87,107,7,-1,Autauga County,Autauga County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
1,62efb19aaf83d320cc1839f3,195,211,1003,1003,AL Baldwin County 01003,13529,14275,102,116,52,9,Baldwin County,Baldwin County,96,98,Housing Choice Vouchers,Housing Choice Vouchers
2,62efb19aaf83d320cc1839f4,178,192,1005,1005,AL Barbour County 01005,12884,13396,72,89,6,6,Barbour County,Barbour County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
3,62efb19aaf83d320cc1839f5,120,158,1009,1009,AL Blount County 01009,12686,14428,82,92,6,9,Blount County,Blount County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
4,62efb19aaf83d320cc1839f6,151,195,1013,1013,AL Butler County 01013,11408,12531,53,65,5,7,Butler County,Butler County,100,100,Housing Choice Vouchers,Housing Choice Vouchers


In [31]:
waits_util_df.dtypes

_id                      object
ave_util_allow_17         int64
ave_util_allow_21         int64
state_county_code        object
code_21                  object
entities                 object
hh_income_17              int64
hh_income_21              int64
months_from_movein_17     int64
months_from_movein_21     int64
months_waiting_17         int64
months_waiting_21         int64
name_17                  object
name_21                  object
pct_utility_allow_17      int64
pct_utility_allow_21      int64
program_label_17         object
program_label_21         object
dtype: object

In [32]:
waits_util_df['state_county_code'] = pd.to_numeric(waits_util_df['state_county_code'], errors='coerce')
waits_util_df.head()

,_id,ave_util_allow_17,ave_util_allow_21,state_county_code,code_21,entities,hh_income_17,hh_income_21,months_from_movein_17,months_from_movein_21,months_waiting_17,months_waiting_21,name_17,name_21,pct_utility_allow_17,pct_utility_allow_21,program_label_17,program_label_21
0,62efb19aaf83d320cc1839f2,162,189,1001.0,1001,AL Autauga County 01001,13903,14185,87,107,7,-1,Autauga County,Autauga County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
1,62efb19aaf83d320cc1839f3,195,211,1003.0,1003,AL Baldwin County 01003,13529,14275,102,116,52,9,Baldwin County,Baldwin County,96,98,Housing Choice Vouchers,Housing Choice Vouchers
2,62efb19aaf83d320cc1839f4,178,192,1005.0,1005,AL Barbour County 01005,12884,13396,72,89,6,6,Barbour County,Barbour County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
3,62efb19aaf83d320cc1839f5,120,158,1009.0,1009,AL Blount County 01009,12686,14428,82,92,6,9,Blount County,Blount County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
4,62efb19aaf83d320cc1839f6,151,195,1013.0,1013,AL Butler County 01013,11408,12531,53,65,5,7,Butler County,Butler County,100,100,Housing Choice Vouchers,Housing Choice Vouchers


In [33]:
rent_df.columns

Index(['County_Name', 'county', 'county_town_name', 'state', 'state_name',
       'State_Alpha', 'Metro_Area_Name', 'ELI_2_17', 'ELI_2_22', 'median2017',
       'median2022', '_id', 'fmr0_17', 'fmr1_17', 'fmr2_17', 'fmr3_17',
       'fmr4_17', 'fmr_0_22', 'fmr_1_22', 'fmr_2_22', 'fmr_3_22', 'fmr_4_22',
       'l50_2_17', 'l50_2_22', 'l80_2_17', 'l80_2_22', 'metro', 'metro_code',
       'state_county_code'],
      dtype='object')

In [35]:
rent_df = pd.merge(rent_df, waits_util_df, on='state_county_code')
rent_df.head()

,County_Name,county,county_town_name,state,state_name,State_Alpha,Metro_Area_Name,ELI_2_17,ELI_2_22,median2017,...,months_from_movein_17_y,months_from_movein_21_y,months_waiting_17_y,months_waiting_21_y,name_17_y,name_21_y,pct_utility_allow_17_y,pct_utility_allow_21_y,program_label_17_y,program_label_21_y
0,Autauga County,1,Autauga County,1,Alabama,AL,"Montgomery, AL MSA",16240,18310,59700,...,87,107,7,-1,Autauga County,Autauga County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
1,Baldwin County,3,Baldwin County,1,Alabama,AL,"Daphne-Fairhope-Foley, AL MSA",16240,20550,61500,...,102,116,52,9,Baldwin County,Baldwin County,96,98,Housing Choice Vouchers,Housing Choice Vouchers
2,Barbour County,5,Barbour County,1,Alabama,AL,"Barbour County, AL",16240,18310,46100,...,72,89,6,6,Barbour County,Barbour County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
3,Blount County,9,Blount County,1,Alabama,AL,"Birmingham-Hoover, AL HUD Metro FMR Area",16240,20400,63100,...,82,92,6,9,Blount County,Blount County,100,100,Housing Choice Vouchers,Housing Choice Vouchers
4,Butler County,13,Butler County,1,Alabama,AL,"Butler County, AL",16240,18310,40900,...,53,65,5,7,Butler County,Butler County,100,100,Housing Choice Vouchers,Housing Choice Vouchers


In [47]:
rent_df['entities_x']

0                               AL Autauga County 01001
1                               AL Baldwin County 01003
2                               AL Barbour County 01005
3                                AL Blount County 01009
4                                AL Butler County 01013
                             ...                       
3940                        PR Villalba Municipio 72149
3941                         PR Yabucoa Municipio 72151
3942                           PR Yauco Municipio 72153
3943     VI St. Croix County, U.S. Virgin Islands 78010
3944    VI St. Thomas County, U.S. Virgin Islands 78030
Name: entities_x, Length: 3945, dtype: object

In [36]:
rent_df.shape

(3945, 63)

In [37]:
rent_df.columns

Index(['County_Name', 'county', 'county_town_name', 'state', 'state_name',
       'State_Alpha', 'Metro_Area_Name', 'ELI_2_17', 'ELI_2_22', 'median2017',
       'median2022', '_id_x', 'fmr0_17', 'fmr1_17', 'fmr2_17', 'fmr3_17',
       'fmr4_17', 'fmr_0_22', 'fmr_1_22', 'fmr_2_22', 'fmr_3_22', 'fmr_4_22',
       'l50_2_17', 'l50_2_22', 'l80_2_17', 'l80_2_22', 'metro', 'metro_code',
       'state_county_code', '_id_y', 'ave_util_allow_17_x',
       'ave_util_allow_21_x', 'code_21_x', 'entities_x', 'hh_income_17_x',
       'hh_income_21_x', 'months_from_movein_17_x', 'months_from_movein_21_x',
       'months_waiting_17_x', 'months_waiting_21_x', 'name_17_x', 'name_21_x',
       'pct_utility_allow_17_x', 'pct_utility_allow_21_x',
       'program_label_17_x', 'program_label_21_x', '_id',
       'ave_util_allow_17_y', 'ave_util_allow_21_y', 'code_21_y', 'entities_y',
       'hh_income_17_y', 'hh_income_21_y', 'months_from_movein_17_y',
       'months_from_movein_21_y', 'months_waiting_17_y',

In [50]:
counts_df = rent_df.copy()
counts_df = counts_df[['program_label_17_y', 'program_label_21_y','program_label_17_x', 'program_label_21_x']]
counts_df.nunique()

program_label_17_y    1
program_label_21_y    1
program_label_17_x    1
program_label_21_x    1
dtype: int64

The columns chosen to be excluded from machine learning and the reasoning for the decision are as follows:


*   County, Metro, State names were dropped as they have numerical counterparts that are representative of them.
*   _id columns were dropped as they are unique identifiers for each data point in the system and thus do not add anything to the machine learning model.
*   Entities were dropped as they are the identifier for the state & county codes concatenated to the area names, all of which is already represented in the dataset. 
*   Program labels were dropped as they are all labeled as "Housing Choice Voucher" as shown above, since there is only one value in this column it does not add anything to the final machine learning model. 

Since there were multiple datasets joined here and in preprocessing of data, there are multiples of many columns for location, name, id, and program labels, so _x and _y as well as 2017 and 2022 version of these columns will need to be dropped to not duplicate data in the final machine learning dataset.



In [52]:
# Drop the non-beneficial ID columns
sec8_df = rent_df.copy()
sec8_df.drop(columns=["County_Name", "Metro_Area_Name","State_Alpha", "_id", "county_town_name","metro_code","state_name", "_id_y", "entities_x", "name_17_x", "name_21_x","code_21_x","program_label_17_x", "program_label_21_x","code_21_y", "entities_y", "program_label_17_y", "program_label_21_y", "name_17_y", "name_21_y"], inplace=True)
sec8_df.head()

,county,state,ELI_2_17,ELI_2_22,median2017,median2022,_id_x,fmr0_17,fmr1_17,fmr2_17,...,ave_util_allow_17_y,ave_util_allow_21_y,hh_income_17_y,hh_income_21_y,months_from_movein_17_y,months_from_movein_21_y,months_waiting_17_y,months_waiting_21_y,pct_utility_allow_17_y,pct_utility_allow_21_y
0,1,1,16240,18310,59700,75500,100199999,587,682,822,...,162,189,13903,14185,87,107,7,-1,100,100
1,3,1,16240,20550,61500,85500,100399999,807,847,977,...,195,211,13529,14275,102,116,52,9,96,98
2,5,1,16240,18310,46100,46400,100599999,501,505,671,...,178,192,12884,13396,72,89,6,6,100,100
3,9,1,16240,20400,63100,84800,100999999,665,751,866,...,120,158,12686,14428,82,92,6,9,100,100
4,13,1,16240,18310,40900,56100,101399999,464,467,621,...,151,195,11408,12531,53,65,5,7,100,100


In [53]:
#check data types are able to be input into a machine learning model
sec8_df.dtypes

county                     int64
state                      int64
ELI_2_17                   int64
ELI_2_22                   int64
median2017                 int64
median2022                 int64
_id_x                      int64
fmr0_17                    int64
fmr1_17                    int64
fmr2_17                    int64
fmr3_17                    int64
fmr4_17                    int64
fmr_0_22                   int64
fmr_1_22                   int64
fmr_2_22                   int64
fmr_3_22                   int64
fmr_4_22                   int64
l50_2_17                   int64
l50_2_22                   int64
l80_2_17                   int64
l80_2_22                   int64
metro                      int64
state_county_code          int64
ave_util_allow_17_x        int64
ave_util_allow_21_x        int64
hh_income_17_x             int64
hh_income_21_x             int64
months_from_movein_17_x    int64
months_from_movein_21_x    int64
months_waiting_17_x        int64
months_wai

In [54]:
#check for null values
sec8_df.isnull().sum(axis=0)

county                     0
state                      0
ELI_2_17                   0
ELI_2_22                   0
median2017                 0
median2022                 0
_id_x                      0
fmr0_17                    0
fmr1_17                    0
fmr2_17                    0
fmr3_17                    0
fmr4_17                    0
fmr_0_22                   0
fmr_1_22                   0
fmr_2_22                   0
fmr_3_22                   0
fmr_4_22                   0
l50_2_17                   0
l50_2_22                   0
l80_2_17                   0
l80_2_22                   0
metro                      0
state_county_code          0
ave_util_allow_17_x        0
ave_util_allow_21_x        0
hh_income_17_x             0
hh_income_21_x             0
months_from_movein_17_x    0
months_from_movein_21_x    0
months_waiting_17_x        0
months_waiting_21_x        0
pct_utility_allow_17_x     0
pct_utility_allow_21_x     0
ave_util_allow_17_y        0
ave_util_allow

In [55]:
#get number of unique categories in columns
sec8_df.nunique()

county                      259
state                        54
ELI_2_17                    138
ELI_2_22                    190
median2017                  527
median2022                  633
_id_x                      3945
fmr0_17                     420
fmr1_17                     443
fmr2_17                     461
fmr3_17                     635
fmr4_17                     703
fmr_0_22                    504
fmr_1_22                    522
fmr_2_22                    556
fmr_3_22                    718
fmr_4_22                    802
l50_2_17                    340
l50_2_22                    393
l80_2_17                    349
l80_2_22                    415
metro                         2
state_county_code          2414
ave_util_allow_17_x         200
ave_util_allow_21_x         201
hh_income_17_x             2029
hh_income_21_x             2016
months_from_movein_17_x     170
months_from_movein_21_x     192
months_waiting_17_x         114
months_waiting_21_x         102
pct_util

In [56]:
sec8_df.describe()

,county,state,ELI_2_17,ELI_2_22,median2017,median2022,_id_x,fmr0_17,fmr1_17,fmr2_17,...,ave_util_allow_17_y,ave_util_allow_21_y,hh_income_17_y,hh_income_21_y,months_from_movein_17_y,months_from_movein_21_y,months_waiting_17_y,months_waiting_21_y,pct_utility_allow_17_y,pct_utility_allow_21_y
count,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3.945000e+03,3945.000000,3945.000000,3945.000000,...,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000,3945.000000
mean,66.708745,30.121673,17473.721166,21121.292776,63775.335868,81732.370089,3.018915e+09,630.975919,703.188593,877.428390,...,126.087199,128.948796,12959.014956,13851.387833,92.756907,110.039544,23.865399,23.953359,83.020786,80.367047
std,93.859078,14.921274,2945.794479,4557.316438,17600.149177,23446.332576,1.493591e+09,187.568815,220.390278,271.643864,...,38.101405,40.229084,3158.366690,3463.354383,33.860223,58.387697,29.855634,21.368812,18.034396,19.965732
min,1.000000,1.000000,5800.000000,5950.000000,15800.000000,21600.000000,1.002000e+08,323.000000,325.000000,432.000000,...,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000,-5.000000
25%,11.000000,22.000000,16240.000000,18310.000000,52200.000000,64800.000000,2.203000e+09,500.000000,554.000000,681.000000,...,101.000000,103.000000,11385.000000,12162.000000,73.000000,80.000000,10.000000,9.000000,75.000000,69.000000
50%,29.000000,26.000000,16240.000000,19350.000000,62200.000000,80100.000000,2.608800e+09,578.000000,633.000000,789.000000,...,127.000000,129.000000,12735.000000,13648.000000,92.000000,100.000000,19.000000,20.000000,88.000000,86.000000
75%,95.000000,41.000000,17800.000000,22700.000000,72600.000000,93500.000000,4.105200e+09,712.000000,817.000000,997.000000,...,149.000000,151.000000,14683.000000,15935.000000,108.000000,119.000000,33.000000,36.000000,97.000000,96.000000
max,840.000000,78.000000,33700.000000,44750.000000,142800.000000,180900.000000,7.803100e+09,1915.000000,2411.000000,3018.000000,...,300.000000,291.000000,23477.000000,26753.000000,792.000000,464.000000,1452.000000,520.000000,100.000000,100.000000


In [57]:
#split preprocessed data into features and target arrays
y = sec8_df.metro
X = sec8_df.drop("metro", 1)

#split preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0, stratify=y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [59]:
#Make sure scaled correctly
X_train.shape

(2958, 42)

In [60]:
X_train_scaled.shape

(2958, 42)

In [61]:
#define the input features for the model
number_input_features = len(X_train.iloc[0])
number_input_features

42

In [62]:
#Create a function to create a new Sequential model with hyperparameter(hp) options
def build_model(hp):
  model = keras.Sequential()
  #allow kerastuner to decide which activation function to use in hidden layers
  activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])


  #allow kerastuner to decide number of neurons in first layer
  model.add(keras.layers.Dense(units=hp.Int('first_units', 
                                            min_value=1,
                                            max_value=10,
                                            step=2), activation=activation, input_dim=number_input_features))
  #allow kerastuner to decide number of hidden layers and neurons in hidden layers
    #using max 2 hidden layers since 3 hidden layers are suspected to be able
    #to appropriately handle any ml model in theory
  for i in range(hp.Int('layers', 1, 2)):
    model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                 min_value=1,
                                                 max_value=10,
                                                 step=2), activation=activation))
    
  model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

  #compile the model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [63]:
#use hyperband (optimized version of random search) to 
  #use early stopping to speed up the hp tuning process
Tuner = kt.Hyperband(build_model,
                     objective="val_accuracy",
                     max_epochs=20,
                     hyperband_iterations=2)

In [64]:
#run kerastuner to find the hp's
Tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 60 Complete [00h 00m 11s]
val_accuracy: 0.8186423778533936

Best val_accuracy So Far: 0.8642350435256958
Total elapsed time: 00h 04m 09s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [65]:
#Get the best model
best_hyper = Tuner.get_best_hyperparameters(1)[0]
best_hyper

In [66]:
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0012',
 'units_0': 9,
 'units_1': 9}

In [67]:
Tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
activation: tanh
first_units: 9
layers: 2
units_0: 9
units_1: 9
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0012
Score: 0.8642350435256958
Trial summary
Hyperparameters:
activation: tanh
first_units: 9
layers: 2
units_0: 7
units_1: 1
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8581560254096985
Trial summary
Hyperparameters:
activation: tanh
first_units: 9
layers: 1
units_0: 9
units_1: 9
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.8530901670455933
Trial summary
Hyperparameters:
activation: tanh
first_units: 9
layers: 1
units_0: 7
units_1: 1
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0042
Score: 0.850050687789917
Trial summary
Hyperparameters:
activation: tanh
first_units: 7
layers: 1
units_0: 7
units_1: 1
tuner/epochs: 20
tuner/

In [68]:
best_model = Tuner.get_best_models(1)[0]


In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

31/31 - 0s - loss: 0.3504 - accuracy: 0.8642 - 265ms/epoch - 9ms/step
Loss: 0.3503709137439728, Accuracy: 0.8642350435256958


In [70]:
#save to a hdf5 file
best_model.save("trained_application_nn.h5")

#import model to new obj
nn_imported = tf.keras.models.load_model('untitled_project/trained_application_nn.h5')

OSError: ignored